### Train, publish and use image detection model with Azure Custom Vision
This is second part which describes how to use API of trained model for object detection.
To see how to use API for upload taged images to Azure Custom Vision Studio project refer to part1 (az_image_detection1.ipynb). <br>
Based on [this](https://microsoftlearning.github.io/mslearn-ai-vision/Instructions/Exercises/03-custom-vision-object-detection.html) Microsoft learn manual.

In [ ]:
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from msrest.authentication import ApiKeyCredentials
from matplotlib import pyplot as plt
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import os
from dotenv import load_dotenv

# Get Configuration Settings
load_dotenv()
prediction_endpoint = os.getenv('PREDICTION_ENDPOINT')
prediction_key = os.getenv('PREDICTION_KEY')
project_id = os.getenv('PROJECT_ID')
model_name = os.getenv('MODEL_NAME')

In [ ]:
# Authenticate a client for the training API
credentials = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key})
prediction_client = CustomVisionPredictionClient(endpoint=prediction_endpoint, credentials=credentials)

In [ ]:
# Load image and get height, width and channels
# data\test-images\produce.jpg
image_url = os.path.join (os.getcwd(), "data", "test-images", "produce.jpg")
print('Detecting objects in file', image_url)
image = Image.open(image_url)
h, w, ch = np.array(image).shape